# Analysis of monthly return data

Description:


In [1]:
import feather
import numpy as np
import pandas as pd

from scipy import sparse

import matplotlib.pyplot as plt

In [2]:
path = '../data/raw/holdings_co_info.feather'
holdings_co_info = feather.read_dataframe(path)
holdings_co_info = holdings_co_info.drop_duplicates()

In [3]:
path = '../data/processed/holdings_summary_s_s.feather'
summary = feather.read_dataframe(path)
summary.shape

(51348, 8)

In [26]:
path = '../data/processed/stock_map.feather'
stock_map = feather.read_dataframe(path)

In [11]:
path = '../data/processed/holdings_b.npz'
holdings = sparse.load_npz(path)
holdings.shape

(159970, 60817)

## Most common stocks

In [12]:
holdings.sum(0).T

matrix([[91.],
        [95.],
        [33.],
        ...,
        [ 2.],
        [ 2.],
        [ 6.]])

In [15]:
holdings_summary = pd.DataFrame(holdings.sum(0).T)
holdings_summary.columns = ['Total']
holdings_summary.shape

(60817, 1)

## Merge Stock_map and Hold_co_info to get name of each security

In [25]:
stock_map['item1'].describe()

count    6.081700e+04
mean     7.331415e+05
std      6.956779e+05
min      0.000000e+00
25%      1.072980e+05
50%      4.816020e+05
75%      1.232904e+06
max      2.382967e+06
Name: item1, dtype: float64

In [19]:
holdings_co_info.sample()

,crsp_company_key,security_name,cusip,permno,permco
888379,4129597.0,FANNIE MAE 01-MAY-2039 AA5795,31416PNM,NaN,NaN


In [ ]:
stock_map_merged = stock_map.merge(holdings_co_info, how='left', left_on = 'crsp_company_key', right_on='crsp_company_key')

In [ ]:
stock_map_merged

In [ ]:
stock_map_merged = pd.concat([holdings_summary,stock_map_merged],axis=1)

In [ ]:
stock_map_merged.loc[stock_map_merged['security_name'].astype('str').str.contains('GAP')]

In [ ]:
stock_map_merged['Total'].describe()

In [ ]:
stock_map_merged.sort_values(by=['Total'],ascending=False).head(100)

## Distribution by date

In [ ]:
# Analysis of selected funds

## Distribution by date(returns['report_dt']
(returns['crsp_fundno']
    .groupby([
         returns['caldt'].dt.year,
#         returns['caldt'].dt.month
     ])
     .count()
     .plot(kind='bar',
           color = 'b',
          figsize=(20,10)))

## Observations per Fund

In [ ]:
# Analysis of selected funds

## Distribution by date(returns['report_dt']
a = (returns['crsp_fundno']
    .groupby([
         returns['crsp_fundno']
     ])
    .count()
)
a = a.groupby(a).count().cumsum().T

In [ ]:
a.plot(kind='line',
       color = 'b',
       figsize=(20,10))

In [ ]:
(returns.loc[returns['crsp_fundno'] == 27650,'mret']
    .plot(kind='bar',
          color = 'b',
          figsize=(20,10)))

## Map Obj codes to returns

In [ ]:
summary.head().dtypes

In [ ]:
returns.head().dtypes

In [ ]:
returns_merged = returns.merge(summary, how='left', on=['crsp_fundno'])
returns_merged = returns_merged.dropna(axis = 0, how = 'any')

In [ ]:
returns_merged.sample(10)

## Aggregate return

In [ ]:
returns_merged.sample(5)

In [ ]:
# Analysis of selected funds
return_class = returns_merged[returns_merged['crsp_obj_cd'] == 'EFSH']

In [ ]:
return_class = (return_class['mret']
    .groupby([
        return_class['crsp_obj_cd'],
        return_class['caldt']
     ])
    .agg(['mean','count']))

In [ ]:
return_class.loc[:,'cumulative'] = (return_class.loc[:,'mean'].fillna(0)+1).cumprod()

Cum return over time

In [ ]:
return_class['mean'].plot(kind='line',
           color = 'b',
          figsize=(16,10))

Different from zero?

In [ ]:
return_class.loc[:,['mean']].boxplot()

## Objective codes

In [ ]:
returns_grouped = (returns_merged['cum']
    .groupby([
        returns_merged['crsp_obj_cd'],
#        returns_merged['caldt']
     ]))

In [ ]:
returns_grouped = returns_grouped.describe()

In [ ]:
returns_grouped['mean'].sort_values().plot(
            kind='bar',
            color = 'b',
            figsize=(16,10))